In [1]:
import sys

sys.path.append("../")
sys.path.append("../raw_to_db/raw_to_json")

import typing as t

import geopandas as gpd
import pandas as pd
from bunka_logger import logger
from data_model import Individual, RawIndividual
from data_model_region import Country
from functions_geo import get_country_model
from sys_utils import load_model
from tqdm import tqdm
from utils import round_nearest

/Users/charlesdedampierre/opt/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/Users/charlesdedampierre/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (
/Users/charlesdedampierre/Desktop/cultura_project/raw_to_db/../raw_to_db/raw_to_json/functions_geo.py:18: FutureWarning: The geopandas.dataset module is deprecated and will be removed in GeoPandas 1.0. You can get the original 'naturalearth_lowres' data from https://www.naturalearthdata.com/downloads/110m-cultural-vectors/.
  df_world = gpd.read_file(gpd.datasets.get_path("naturalearth_lowres"))


In [2]:
raw_individuals: t.List[RawIndividual] = load_model(
    RawIndividual, name="../raw_data/checkpoints_dev_2/checkpoint_1.jsonl"
)

In [22]:
from data_model_region import Region
from functions_regions import  get_regions
from functions_regions import get_individuals_regions

CHECKPOINT_PATH = "../raw_data/checkpoints_dev_2/"

individuals = load_model(
    Individual, name=CHECKPOINT_PATH + "/checkpoint_3.jsonl"
)

regions: t.List[Region] = get_regions()

In [23]:
individuals: t.List[Individual] = get_individuals_regions(individuals, regions)

/Users/charlesdedampierre/Desktop/cultura_project/raw_to_db/../raw_to_db/raw_to_json/functions_regions.py:168: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_reg["range"] = df_reg.apply(
/Users/charlesdedampierre/Desktop/cultura_project/raw_to_db/../raw_to_db/raw_to_json/functions_regions.py:171: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_reg["impact_year"] = df_reg["range"].apply(
100%|██████████| 180635/180635 [00:02<00:00, 71164.00it/s]


In [27]:
test = [x for x in individuals if x.id.wikidata_id == 'Q337126']


[Individual(id=RawIndividual(wikidata_id='Q337126', name='Al-Bakri', birthyear=1014, gender=['male'], raw_nationalities=[RawNationality(wikidata_id='Q171740', name='Caliphate of Córdoba', location='Point(-4.76666667 37.88333333)'), RawNationality(wikidata_id='Q276951', name='Taifa of Seville', location='nan'), RawNationality(wikidata_id='Q2285679', name='Taifa of Córdoba', location='Point(-4.76667 37.8833)')], raw_birthcities=[RawBirthcity(wikidata_id='Q12246', name='Huelva', location='Point(-6.95 37.25)', country_wikidata_id='Q29', country_name='Spain', country_location='Point(-3.5 40.2)')], raw_deathcities=[RawDeathcity(wikidata_id='Q5818', name='Córdoba', location='Point(-4.766666666 37.883333333)', country_wikidata_id='Q29', country_name='Spain', country_location=None)], occupations=[Occupation(wikidata_id='Q901402', name='geographer', category=['science', 'writer']), Occupation(wikidata_id='Q2374149', name='botanist', category=['science', 'writer']), Occupation(wikidata_id='Q20178

In [4]:
from data_model import Individual, RawIndividual
from functions_enrich_wikidata import raw_to_individuals

In [5]:
individuals: t.List[Individual] = raw_to_individuals(raw_individuals)

In [7]:
individuals[0].__fields__

{'id': ModelField(name='id', type=RawIndividual, required=True),
 'impact_years': ModelField(name='impact_years', type=Optional[Tuple[int, int]], required=False, default=None),
 'cultural_score': ModelField(name='cultural_score', type=Optional[float], required=False, default=None),
 'country': ModelField(name='country', type=Optional[Country], required=False, default=None),
 'wikipedia_pages': ModelField(name='wikipedia_pages', type=Optional[List[WikipediaPage]], required=False, default=None),
 'regions': ModelField(name='regions', type=Optional[List[str]], required=False, default=None),
 'identifiers': ModelField(name='identifiers', type=Optional[List[ExternalID]], required=False, default=None),
 'country_data_origin': ModelField(name='country_data_origin', type=Optional[str], required=False, default=None)}

In [8]:
import typing as t

import geopandas as gpd
import pandas as pd
from bunka_logger import logger
from data_model import Individual, RawIndividual
from data_model_region import Country
from functions_geo import get_country_model
from sys_utils import load_model
from tqdm import tqdm
from utils import round_nearest

tqdm.pandas()

In [9]:
# Birthcities
individual_birthcities = [
    {
        "individual_wikidata_id": x.id.wikidata_id,
        "location": [y.location for y in x.id.raw_birthcities],
    }
    for x in individuals
    if x.id.raw_birthcities != None
]

df_individual_location = pd.DataFrame(individual_birthcities)
df_individual_location = df_individual_location.explode("location")
df_individual_location = df_individual_location[
    df_individual_location["location"] != "nan"
]
df_individual_location = df_individual_location.dropna()

df_unique_locations = df_individual_location[["location"]].drop_duplicates()

df_unique_locations["country_model"] = df_unique_locations[
    "location"
].progress_apply(lambda x: get_country_model(x))
df_unique_locations = df_unique_locations.dropna()

df_unique_locations["country_name"] = df_unique_locations["country_model"].apply(
    lambda x: x.name
)

100%|██████████| 32565/32565 [00:25<00:00, 1278.82it/s]


In [10]:
df_final = pd.merge(df_individual_location, df_unique_locations, on="location")

df_final_birthcities = df_final.drop_duplicates(
    "individual_wikidata_id", keep="first"
)
df_final_birthcities = df_final_birthcities[
    ["individual_wikidata_id", "country_name"]
].drop_duplicates()
df_final_birthcities["origin"] = "birthcity"

In [11]:

# Deathcities (There was an issue regarding the loading, hence the diffenrent way of loading)
df_individual_location = pd.DataFrame(
    {
        "individual_wikidata_id": [x.id.wikidata_id for x in individuals],
        "deathcities": [x.id.raw_deathcities for x in individuals],
    }
)
df_individual_location = df_individual_location.dropna()
df_individual_location = df_individual_location.explode("deathcities")

def get_location_point(x):
    try:
        return x.location
    except:
        return None

df_individual_location["location"] = df_individual_location["deathcities"].apply(
    lambda x: get_location_point(x)
)
df_individual_location = df_individual_location.dropna()
df_individual_location = df_individual_location.drop("deathcities", axis=1)
df_individual_location = df_individual_location.drop_duplicates()
df_individual_location = df_individual_location.reset_index(drop=True)

df_unique_locations = df_individual_location[["location"]].drop_duplicates()

df_unique_locations["country_model"] = df_unique_locations[
    "location"
].progress_apply(lambda x: get_country_model(x))
df_unique_locations = df_unique_locations.dropna()

df_unique_locations["country_name"] = df_unique_locations["country_model"].apply(
    lambda x: x.name
)

df_final = pd.merge(df_individual_location, df_unique_locations, on="location")
df_final_deathcities = df_final.drop_duplicates(
    "individual_wikidata_id", keep="first"
)
df_final_deathcities = df_final_deathcities[
    ["individual_wikidata_id", "country_name"]
].drop_duplicates()
df_final_deathcities["origin"] = "deathcity"


100%|██████████| 18938/18938 [00:15<00:00, 1234.22it/s]


In [12]:
# Nationality
individual_nationalities = [
    {
        "individual_wikidata_id": x.id.wikidata_id,
        "location": [y.location for y in x.id.raw_nationalities],
    }
    for x in individuals
    if x.id.raw_nationalities != None
]
df_individual_location_nationality = pd.DataFrame(individual_nationalities)
df_individual_location_nationality = df_individual_location_nationality.explode(
    "location"
)

df_unique_locations_nationality = df_individual_location_nationality[
    ["location"]
].drop_duplicates()
df_unique_locations_nationality["country_model"] = df_unique_locations_nationality[
    "location"
].progress_apply(lambda x: get_country_model(x))
df_unique_locations_nationality = df_unique_locations_nationality.dropna()

df_unique_locations_nationality["country_name"] = df_unique_locations_nationality[
    "country_model"
].apply(lambda x: x.name)

df_final_nationality = pd.merge(
    df_unique_locations_nationality,
    df_individual_location_nationality,
    on="location",
)
df_final_nationality["origin"] = "nationality"
df_final_nationality = df_final_nationality[
    ["individual_wikidata_id", "country_name", "origin"]
].drop_duplicates()



100%|██████████| 604/604 [00:00<00:00, 1328.16it/s]


In [13]:
# Merging the three datasets
final = pd.concat(
    [df_final_deathcities, df_final_nationality, df_final_birthcities]
)
order = ["deathcity", "birthcity", "nationality"]  # order to chose from

sorting_key = lambda x: order.index(x) if x in order else float("inf")

# Sort the DataFrame based on the 'origin' column using the custom key
final = final.sort_values(by="origin", key=lambda x: x.map(sorting_key))

final = final.drop_duplicates("individual_wikidata_id", keep="first")
final = final.reset_index(drop=True)
final = final.drop_duplicates()

df_world = gpd.read_file(gpd.datasets.get_path("naturalearth_lowres"))
df_world["country_model"] = df_world.apply(
    lambda x: Country(name=x["name"], iso_a3=x["iso_a3"]), axis=1
)
df_world = df_world[["name", "country_model"]]
df_world = df_world.rename(columns={"name": "country_name"})

final = pd.merge(df_world, final, on="country_name")
#final = final.rename(columns={"origin": "country_data_origin"})

/var/folders/lx/9b13r6ln63jb54_3xlf9b_380000gn/T/ipykernel_65211/1418879997.py:16: FutureWarning: The geopandas.dataset module is deprecated and will be removed in GeoPandas 1.0. You can get the original 'naturalearth_lowres' data from https://www.naturalearthdata.com/downloads/110m-cultural-vectors/.
  df_world = gpd.read_file(gpd.datasets.get_path("naturalearth_lowres"))


In [14]:
df_final_deathcities[df_final_deathcities['individual_wikidata_id']=='Q37577']

,individual_wikidata_id,country_name,origin
53942,Q37577,France,deathcity


In [15]:
final[final['individual_wikidata_id']=='Q37577']

,country_name,country_model,individual_wikidata_id,origin
33078,France,name='France' iso_a3='FRA',Q37577,deathcity


In [17]:

individual_country = final[["individual_wikidata_id", "country_model"]].to_dict(
    orient="records"
)
dict_individual_country = {
    x["individual_wikidata_id"]: x["country_model"] for x in individual_country
}

individual_country_origin = final[["individual_wikidata_id", "origin"]].to_dict(
    orient="records"
)

dict_individual_country_origin = {
    x["individual_wikidata_id"]: x["origin"] for x in individual_country_origin
}

# Insert to the model
new_individuals = []

for ind in individuals:
    ind.country = dict_individual_country.get(ind.id.wikidata_id)
    ind.country_data_origin =  dict_individual_country_origin.get(ind.id.wikidata_id)
    new_individuals.append(ind)

Individual(id=RawIndividual(wikidata_id='Q91335', name='Albert Fraenkel', birthyear=1848, gender=['male'], raw_nationalities=[RawNationality(wikidata_id='Q183', name='Germany', location='Point(10.0 51.0)')], raw_birthcities=[RawBirthcity(wikidata_id='Q4024', name='Frankfurt (Oder)', location='Point(14.551667 52.341944)', country_wikidata_id='Q183', country_name='Germany', country_location='Point(10.0 51.0)')], raw_deathcities=[RawDeathcity(wikidata_id='Q64', name='Berlin', location='Point(13.383333333 52.516666666)', country_wikidata_id='Q183', country_name='Germany', country_location=None)], occupations=[Occupation(wikidata_id='Q1622272', name='university teacher', category=['writer'])]), impact_years=None, cultural_score=None, country=Country(name='Germany', iso_a3='DEU'), wikipedia_pages=None, regions=None, identifiers=None, country_data_origin='deathcity')

In [16]:
new_individuals = get_country_code(individuals)


100%|██████████| 604/604 [00:00<00:00, 1413.28it/s]
/var/folders/lx/9b13r6ln63jb54_3xlf9b_380000gn/T/ipykernel_54533/1759187402.py:131: FutureWarning: The geopandas.dataset module is deprecated and will be removed in GeoPandas 1.0. You can get the original 'naturalearth_lowres' data from https://www.naturalearthdata.com/downloads/110m-cultural-vectors/.
  df_world = gpd.read_file(gpd.datasets.get_path("naturalearth_lowres"))


[Individual(id=RawIndividual(wikidata_id='Q5223046', name='Dariyavji', birthyear=1676, gender=['male'], raw_nationalities=None, raw_birthcities=None, raw_deathcities=None, occupations=[Occupation(wikidata_id='Q49757', name='poet', category=['writer'])]), impact_years=None, cultural_score=None, country=None, wikipedia_pages=None, regions=None, identifiers=None),
 Individual(id=RawIndividual(wikidata_id='Q249119', name='Wilhelm Victor Alfred Tepe', birthyear=1840, gender=['male'], raw_nationalities=[RawNationality(wikidata_id='Q29999', name='Kingdom of the Netherlands', location='Point(4.883333333 52.366666666)')], raw_birthcities=[RawBirthcity(wikidata_id='Q727', name='Amsterdam', location='Point(4.9 52.383333333)', country_wikidata_id='Q55', country_name='Netherlands', country_location='Point(5.55 52.316666666)')], raw_deathcities=[RawDeathcity(wikidata_id='Q1718', name='Düsseldorf', location='Point(6.772380555 51.231144444)', country_wikidata_id='Q183', country_name='Germany', country

In [22]:
new_individuals[2].country

Country(name='Germany', iso_a3='DEU')